In [ ]:
import api_IBGE as ibge

## Dados da PIA

In [ ]:
#checa metadados da PIA, sem especificar periodicidade não tras todos os dados
ibge.consulta_metadados(tabela='2221')

In [ ]:
df = ibge.consulta_ibge(tabela='2221',
                        periodos='1966|1967|1968|1969|1973|1974|1976|1977|1978|1979|1981|1982|1983|1984|1988|1989|1990|1992|1993|1994|1995',
                   variaveis='1150|1151|215|811', 
                   classificacao_categorias='12123[101229,101230]',
                   localidades='N1[all]')

df = ibge.limpa_resultado(df)

In [ ]:
df.tail(50)

In [ ]:
df.to_csv('pesquisa_industrial_anual.csv')

### Ajuste das moedas

In [ ]:
import pandas as pd
moedas_brasil = {
    (1500, 1833): "Real português (Réis)",
    (1833, 1942): "Real brasileiro (Réis)",
    (1942, 1967): "Cruzeiro (Cr$)",
    (1967, 1986): "Cruzeiro Novo (NCr$)",
    (1986, 1989): "Cruzado (Cz$)",
    (1989, 1990): "Cruzado Novo (NCz$)",
    (1990, 1993): "Cruzeiro (Cr$)",
    (1993, 1994): "Cruzeiro Real (CR$)",
    (1994, 2025): "Real (R$)"
}


def obter_moeda(ano):
    for intervalo, nome in moedas_brasil.items():
        if intervalo[0] <= ano < intervalo[1]:
            return nome
    return "Moeda desconhecida"

# Aplicar a conversão ao DataFrame
df.loc[42:,'Unidade'] = df.loc[42:,'ano'].astype(int).apply(obter_moeda)



In [ ]:
df.tail(50)

In [ ]:
df.to_csv('Dados/pesquisa_industrial_anual_1966_1995.csv')

## Dados Contas Nacionais

### Dados pós-1989
fonte -> https://seculoxx.ibge.gov.br/economicas/contas-nacionais


### Dados pré-1989
fonte ->https://seculoxx.ibge.gov.br/economicas/contas-nacionais